# Making Recommendations Based on Popularity

These datasets are hosted on: https://archive.ics.uci.edu/ml/datasets/Restaurant+%26+consumer+data

They were originally published by: Blanca Vargas-Govea, Juan Gabriel González-Serna, Rafael Ponce-Medellín. Effects of relevant contextual features in the performance of a restaurant recommender system. In RecSys11: Workshop on Context Aware Recommender Systems (CARS-2011), Chicago, IL, USA, October 23, 2011.

## Restaurants data

In [1]:
import numpy as np
import pandas as pd

In [2]:
# rating_final.csv
url = 'https://drive.google.com/file/d/1ptu4AlEXO4qQ8GytxKHoeuS1y4l_zWkC/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
frame = pd.read_csv(path)

# chefmozcuisine.csv
url = 'https://drive.google.com/file/d/1S0_EGSRERIkSKW4D8xHPGZMqvlhuUzp1/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
cuisine = pd.read_csv(path)

# 'geoplaces2.csv'
url = 'https://drive.google.com/file/d/1ee3ib7LqGsMUksY68SD9yBItRvTFELxo/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
geodata = pd.read_csv(path, encoding = 'CP1252') # change encoding to 'mbcs' in Windows

On the "frame" dataset we have the ratings users have given to places. Ratings go from 0 to 2.

In [3]:
frame.head(3)

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2


In the `geodata` dataset we have info about the places. We will only use the `name` column.

In [4]:
geodata.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [5]:
places =  geodata[['placeID', 'name']]
places.head()

,placeID,name
0,134999,Kiku Cuernavaca
1,132825,puesto de tacos
2,135106,El Rincón de San Francisco
3,132667,little pizza Emilio Portes Gil
4,132613,carnitas_mata


In the `cuisine` dataset we have the type of cuisine that restaurants offer.

In [6]:
cuisine.head(3)

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American


## Popularity/Quality based recommmender system

Let's group places by rating, and look at their average rating. This is an **explicit** rating given by users.

In [7]:
frame.groupby('placeID')['rating'].mean()

placeID
132560    0.500000
132561    0.750000
132564    1.250000
132572    1.000000
132583    1.000000
            ...   
135088    1.000000
135104    0.857143
135106    1.200000
135108    1.181818
135109    1.000000
Name: rating, Length: 130, dtype: float64

In [8]:
rating = pd.DataFrame(frame.groupby('placeID')['rating'].mean())
rating.sort_values("rating", ascending=False).head()

,rating
placeID,
132955,2.000000
135034,2.000000
134986,2.000000
132922,1.833333
132755,1.800000


The top rated places have a perfect score of 2/2. But how many reviews do these places have?

In [9]:
frame.query("placeID==132955")

,userID,placeID,rating,food_rating,service_rating
934,U1004,132955,2,2,2
960,U1061,132955,2,2,2
996,U1059,132955,2,1,2
1014,U1097,132955,2,2,1
1080,U1096,132955,2,2,2


Looks like only 5 people went to this place. Maybe they're just the owner's friends! Or maybe they're really top-quality places, but too niche to recommend to the masses.

We can also look at how many times each restaurant has received a rating. The ratings count is an **implicit** rating.

In [10]:
rating['rating_count'] = frame.groupby('placeID')['rating'].count()
rating.sort_values("rating_count", ascending=False).head()

,rating,rating_count
placeID,,
135085,1.333333,36
132825,1.281250,32
135032,1.178571,28
135052,1.280000,25
132834,1.000000,25


Some places have been visited around 30 times. They are more popular than the top rated places, but received lower explicit ratings.

Let's locate the most popular place, and get some info about it:

In [11]:
rating.sort_values('rating_count', ascending=False).head()

,rating,rating_count
placeID,,
135085,1.333333,36
132825,1.281250,32
135032,1.178571,28
135052,1.280000,25
132834,1.000000,25


In [12]:
rating.sort_values('rating_count', ascending=False).head(1).index[0]

135085

In [13]:
# placeId of most popular place
top_popular_placeID = rating.sort_values('rating_count', ascending=False).head(1).index[0]

# name of the most popular place
places[places['placeID']==top_popular_placeID]

,placeID,name
121,135085,Tortas Locas Hipocampo


In [14]:
# cuisine of the most popular place
cuisine[cuisine['placeID']==top_popular_placeID]

,placeID,Rcuisine
44,135085,Fast_Food


The most popular place is "Tortas Locas Hipocampo", a fast food place that has received 36 reviews and it has an average score of 1.33.

### Challenge 1:

Find a hybrid system to sort restaurants, so that you can recommend the "best" places: restaurants that are both high rated and popular.

Create a new column that represents new ranking of both columns ratings and ratings_count e.g., rating * (ratings_count /5)

Join with correpsonding table to show the name of the resturent.


In [15]:
def get_n_resturents(rating = None, n = 10):
  if rating is None:
    return 'sorry you missed the dataframe'
  rating['overall_rating'] = rating['rating'] * (rating['rating_count'] * 0.25)
  return rating.sort_values(by='overall_rating', ascending=False).head(n)

get_n_resturents()

'sorry you missed the dataframe'

### Challenge 2:

Find a hybrid system to sort movies, so that you can recommend the "best" movies: movies that are both high rated and popular.

Join tables to show he name of the movies

In [16]:
import pandas as pd

url = 'https://drive.google.com/file/d/1S0CtDB8NYUs94KgO0VDv6b2R1CShQcLF/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
links = pd.read_csv(path)


url = 'https://drive.google.com/file/d/1sW3zww6gMzoln0-U0Zs7HW_bKYjtH99i/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
movies = pd.read_csv(path)

url = 'https://drive.google.com/file/d/1nUpoWkhzhnYtUFvGYTR317RHiq7XtTx9/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
ratings = pd.read_csv(path)

url = 'https://drive.google.com/file/d/1F9szBIzHvE9sk-p89sk1zpxVEG_gJezg/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
tags = pd.read_csv(path)

In [17]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [18]:
score_results = pd.DataFrame(ratings.groupby('movieId')['rating'].mean())


In [19]:
score_results['rating_count'] = ratings.groupby('movieId')['rating'].count()

In [20]:
score_results

,rating,rating_count
movieId,,
1,3.920930,215
2,3.431818,110
3,3.259615,52
4,2.357143,7
5,3.071429,49
...,...,...
193581,4.000000,1
193583,3.500000,1
193585,3.500000,1


In [21]:
def get_n_movies(score_results, n):
  score_results['overall_rating'] = score_results['rating'] * (score_results['rating_count'] * 0.1)
  return score_results.sort_values(by='overall_rating', ascending=False).head(n)

movies_ranked  = get_n_movies(score_results, 5)  

In [23]:
movies_ranked

,rating,rating_count,overall_rating
movieId,,,
318,4.429022,317,140.40
356,4.164134,329,137.00
296,4.197068,307,128.85
2571,4.192446,278,116.55
593,4.161290,279,116.10


In [22]:
df = (movies_ranked.reset_index(level = 0))
df.merge(movies, how='inner', left_on = 'movieId', right_on = 'movieId') # == df.merge(movies) both will do the same

,movieId,rating,rating_count,overall_rating,title,genres
0,318,4.429022,317,140.40,"Shawshank Redemption, The (1994)",Crime|Drama
1,356,4.164134,329,137.00,Forrest Gump (1994),Comedy|Drama|Romance|War
2,296,4.197068,307,128.85,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,2571,4.192446,278,116.55,"Matrix, The (1999)",Action|Sci-Fi|Thriller
4,593,4.161290,279,116.10,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller


In [24]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [25]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
